In [ ]:
# DataFrame, Stream

In [1]:
import findspark
findspark.init()
findspark.find()

'/Users/chiholee/02.Project/spark/spark'

In [2]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = ""
os.environ['AWS_SECRET_ACCESS_KEY'] = ""
os.environ['AWS_DEFAULT_REGION'] = ""

In [3]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("Read S3 Parquet") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/06 10:24:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.conf.set("spark.hadoop.fs.s3a.endpoint", "s3.ap-northeast-2.amazonaws.com")
spark.conf.set("spark.hadoop.fs.s3a.impl",
               "org.apache.hadoop.fs.s3a.S3AFileSystem")


In [5]:
df = spark.read \
    .format("parquet") \
    .option("header", True) \
    .load("s3a://seminar-s3/accesslog/2023/04/06/00/")
    

23/04/06 10:24:23 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
dataSchema = df.schema
dataSchema

StructType([StructField('timestamp', TimestampType(), True), StructField('client_ip', StringType(), True), StructField('cust_id', LongType(), True), StructField('prd_id', LongType(), True), StructField('service', StringType(), True), StructField('http_referer', StringType(), True), StructField('dt', StringType(), True)])

In [7]:
df = spark.read \
    .format("parquet") \
    .option("header", True) \
    .load("s3a://seminar-s3/accesslog/2023/04/06/01/")

df.createOrReplaceTempView("df_accesslog")

# spark.sql("""
# select service customer_click, count(*) cnt
# from df_accesslog
# group by service
# order by cnt
# """).show()

spark.sql("""
select prd_id, service, count(*) cnt
from df_accesslog
where prd_id <= 10
and service = 'order'
group by prd_id, service
order by cnt desc;
""").show()


+------+-------+---+
|prd_id|service|cnt|
+------+-------+---+
|     8|  order|666|
|     6|  order|663|
|     5|  order|660|
|     4|  order|658|
|     1|  order|652|
|     7|  order|645|
|     2|  order|637|
|     3|  order|632|
|     9|  order|623|
+------+-------+---+



In [8]:
import time
from IPython.display import display, clear_output

sql_title = "sql_14"

streaming = spark.readStream\
                .schema(dataSchema)\
                .load("s3a://seminar-s3/accesslog/2023/04/06/01/")

streaming.createOrReplaceTempView("stream_accesslog")

# spark.conf.set("spark.sql.adaptive.enabled", False)

test_01 = spark.sql("""
select prd_id, service, count(*) cnt
from stream_accesslog
where prd_id <= 10
and service = 'order'
group by prd_id, service
order by cnt desc
""").writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName(sql_title) \
    .start()
# .start().awaitTermination()



23/04/06 10:24:37 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/wg/dx6n4cld3kg_d3gcthzc01km0000gr/T/temporary-ca5baee5-c25e-44f5-96d7-eb9fee8246fa. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/06 10:24:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [9]:

while True:
    clear_output(wait=True)
    display(spark.sql('SELECT * FROM '+sql_title).show())
    time.sleep(5)


+------+-------+---+
|prd_id|service|cnt|
+------+-------+---+
|     4|  order|725|
|     6|  order|721|
|     8|  order|718|
|     5|  order|714|
|     1|  order|707|
|     3|  order|692|
|     2|  order|690|
|     7|  order|686|
|     9|  order|678|
+------+-------+---+



None

KeyboardInterrupt: 